<a href="https://colab.research.google.com/github/sir-sauc3/comp215-Project1/blob/main/Sos%26Roberto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# this is our key: k_w26fb6h9
# api url top 250 movies: https://imdb-api.com/en/API/Top250Movies/k_w26fb6h9
# api url for movie casst: https://imdb-api.com/en/API/FullCast/k_w26fb6h9/${movie ID}


In [10]:
import json, requests
from pprint import pprint
from dataclasses import dataclass

In [11]:
@dataclass
class Actor:
  name : str
  id : str
  image : str

  @classmethod
  def createActor(cls, name, id, image):
    return cls(name, id, image)
    

@dataclass
class Movie:
  title : str
  id : str
  image: str

  @classmethod
  def createMovie(cls, title, id, image):
    return cls(title, id, image)

In [15]:
API_KEY = 'k_w26fb6h9'

url = f'https://imdb-api.com/en/API/Top250Movies/k_w26fb6h9'

response = requests.request("GET", url, headers={}, data={})

data = json.loads(response.text)
pprint(data)

movies = [(movie['title'],
          movie['id'],
          movie['image']) for movie in data['items']]
# pprint(movies)


# movies = [[data['items'][i]['title'], 
#            data['items'][i]['id'], 
#            [data['items'][i]['image']]
#            ] for i in range(len(data['items']))]

{'errorMessage': 'Maximum usage (110 of 100 per day)', 'items': []}


In [21]:
movie_id = 'tt1950186'

url = f'https://imdb-api.com/en/API/FullCast/k_w26fb6h9/{movie_id}'

response = requests.request("GET", url, headers={}, data={})

data = json.loads(response.text)
# pprint(data)

fullcast=[]
fullcast = [(actor['name'],
            actor['id'],
            actor['image']) for actor in data['actors']]
pprint(fullcast)

"""
NOTED THAT FULLCAST ABOVE CONTAINS THE CAST OF ONE MOVIE ONLY. HOWEVER,
FULLCAST IN THE CODE BELOW CONTAINS A LIST OF DICTIONARIES (MOVIE ID BEING KEY AND LIST OF ACTORS BEING VALUE)
OF ALL THE CAST MEMBERS FROM THE APIs DATABASE
MEANING IT MAKES A TOTAL OF 250 REQUESTS
"""


# fullcast = [dict(id=[(cast_member['name'], 
#                       cast_member['id'], 
#                       cast_member['image']) 
#                       for cast_member in json.loads(requests.request("GET",
#                                                                      f'https://imdb-api.com/en/API/FullCast/k_w26fb6h9/{id}', 
#                                                                      headers={}, 
#                                                                      data={}).text)['actors']])
#                                                                      for (title, id, image) in movies]


# no_picture = len([1 for tup in fullcast if 'nopicture' in tup[2]])
# no_picture

# fullcast = [[data['actors'][i]['name'], 
#              data['actors'][i]['id'], 
#              data['actors'][i]['image']] 
#             for i in range(len(data['actors']))]

TypeError: ignored

In [20]:
# Movielist = [Movie.createMovie(movieInfo[0], movieInfo[1], movieInfo[2]) for movieInfo in movies]
# pprint(Movielist)

# Actorlist = [Actor.createActor(actorInfo[0], actorInfo[1], actorInfo[2]) for actorInfo in fullcast]
# pprint(Actorlist)


Movielist = [Movie.createMovie(title, id, image) for (title, id, image) in movies]
pprint(Movielist)

Actorlist = [Actor.createActor(name, id, image) for (name, id, image) in fullcast]
pprint(Actorlist)

[]
[]
